### Embed data with joint LSTM

Embed the downstream data using the joint LSTM trained on all signals simultaneously.

## Embed the downstream prediction data

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
import sys
sys.path.insert(0,"..")
from embedding_utils import *
import numpy as np

Using TensorFlow backend.
W0213 09:45:03.707623 140235354949440 deprecation_wrapper.py:119] From ../embedding_utils.py:16: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0213 09:45:03.709717 140235354949440 deprecation_wrapper.py:119] From ../embedding_utils.py:16: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.

W0213 09:45:03.711277 140235354949440 deprecation_wrapper.py:119] From ../embedding_utils.py:17: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



### Generate embeddings for nextfivemulti etco2 hosp_model 1

In [ ]:
import sys
sys.path.append("..")
from xgb_setup import *
os.nice(5)
# PATH = "/projects/leelab2/hughchen/RELIC/repr_learning/"
PATH = "/homes/gws/hughchen/phase/downstream_prediction/"
RESULTPATH = PATH+"/results/"; MODELPATH = PATH+"/models/"
lookback = 60
DEBUG = False

label_type_eta_currfeat_lst = [("etco235",0.1,"ETCO2")]

for label_type, _, curr_feat in label_type_eta_currfeat_lst:
    print("\n[Progress] label_type: {}, eta: {}, curr_feat {}".format(label_type, "NA", curr_feat))

    mod_type = "lstmarms_{}_top15".format(label_type)
    RESDIR = '{}{}/'.format(RESULTPATH, mod_type)
    if not os.path.exists(RESDIR): os.makedirs(RESDIR)

    for hosp_model in [1]:

        # This is the order of features that the big LSTM uses
        top15 = ["SAO2","FIO2","ECGRATE","ETCO2","RESPRATE","PEEP","TV","PEAK",
                 "PIP","ETSEVO","ETSEV","TEMP1","NIBPM","NIBPS","NIBPD"]

        MODDIR = "/homes/gws/hughchen/phase/upstream_embedding/models/"
        MODDIR += "multivariate_biglstmdropout_hd{}_nextfivetask_".format(hosp_model)
        MODDIR += "200n_200n_200ep_rmspropopt_0.001lr_0.5drop_1000bs_epochsize"
        min_mod = load_min_model_helper(MODDIR)
        min_mod_weights = min_mod.get_weights()

        for hosp_data in [1]:
            print("\n[Progress] hosp_data {}".format(hosp_data))

            data_type = "proc[top15]+nonsignal"

            print("\n[Progress] data_type {}".format(data_type))
            # Load train validation data and split it
            (X_trval_lst,y_trval) = load_data(PATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
            X_trval_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_trval_lst]
            
            # Load test data
            (X_test1_lst,y_test1) = load_data(PATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
            X_test1_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_test1_lst]
            
            # Create and save embeddings for top15 features
            task = "nextfivemulti"
            suffix = "embedding_data{}_model{}.npy".format(hosp_data,hosp_model)
            final_task = label_type
            SPATH = PATH+"data/{}/hospital_{}/hidden200/{}/model_{}/".format(final_task,hosp_data,task,hosp_model)
            if not os.path.exists(SPATH): os.makedirs(SPATH)

            for i in range(0,len(top15)):
                feat = top15[i] # Current feature being embedded
                
                # Create model
                h1 = 200; h2 = 200; lookback = 60
                model = Sequential()
                model.add(LSTM(h1, recurrent_dropout=0.5, return_sequences=True, input_shape=(lookback,1), 
                                weights=min_mod_weights[(i*3):((i+1)*3)]))
                model.add(LSTM(h2, recurrent_dropout=0.5,dropout=0.5,
                               weights=min_mod_weights[(45+(i*3)):(45+((i+1)*3))]))

                # Create embeddings and names
                trval_pred = model.predict(X_trval_lst[i])
                test1_pred = model.predict(X_test1_lst[i])
                trval_pred_name = "task:{}_feat:{}_trval_{}".format(task,feat,suffix)
                test1_pred_name = "task:{}_feat:{}_test1_{}".format(task,feat,suffix)

                # Save embeddings
                np.save(SPATH+trval_pred_name,trval_pred)
                np.save(SPATH+test1_pred_name,test1_pred)


[Progress] label_type: etco235, eta: NA, curr_feat ETCO2
[PROGRESS] Starting load_min_model_helper()
[DEBUG] MPATH /homes/gws/hughchen/phase/upstream_embedding/models/multivariate_biglstmdropout_hd1_nextfivetask_200n_200n_200ep_rmspropopt_0.001lr_0.5drop_1000bs_epochsize


W0213 09:45:15.100636 140235354949440 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0213 09:45:15.184010 140235354949440 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



[DEBUG] min_mod_name val_loss:10.944383427112342_epoch:177_multivariate_biglstmdropout_hd1_nextfivetask_200n_200n_200ep_rmspropopt_0.001lr_0.5drop_1000bs_epochsize.h5


W0213 09:45:15.612288 140235354949440 deprecation.py:506] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0213 09:45:59.127630 140235354949440 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0213 09:46:04.730756 140235354949440 deprecation.py:323] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Us


[Progress] hosp_data 1

[Progress] data_type proc[top15]+nonsignal
[DEBUG] Starting load_raw_data
[DEBUG] DPATH /homes/gws/hughchen/phase/downstream_prediction//data/etco235/hospital_1/
[DEBUG] Starting load_raw_data
[DEBUG] DPATH /homes/gws/hughchen/phase/downstream_prediction//data/etco235/hospital_1/
